In [45]:
RAW_FILE_PATH = '../../data/raw/Transaction.csv'
OUTPUT_PATH = '../../data/'

In [47]:
%run '../schema_definition/schema.py'

In [48]:
import sys
sys.path.insert(0, '../../')

from schema_definition.schema import *

In [49]:
from pyspark.shell import spark
from pyspark.sql import functions as F

In [51]:
gold_df = spark.read.json(OUTPUT_PATH + "gold", gold_schema)
calculate_response = gold_df.withColumn("Response", F.datediff(F.col("RequestDate"), F.col("ImplementedDate")))
calculate_response.createOrReplaceTempView("fastest_response")

fastest_response_query = """
                        SELECT *
                        FROM fastest_response
                        ORDER BY Response DESC
                        """
fastest_response_df = spark.sql(fastest_response_query)
fastest_response_df.coalesce(1).write.csv(OUTPUT_PATH + 'response', sep=',', header=True, mode='overwrite')
# fastest_response_df.printSchema()
# fastest_response_df.show(5, truncate=False)
# import databricks.koalas as ks
# koalas_df = ks.DataFrame(fastest_response_df)
# print(koalas_df)